In [ ]:
!pip install nltk
!pip install category_encoders

In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import unicodedata
from pymorphy3 import MorphAnalyzer
import nltk
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import OneHotEncoder
from category_encoders import HashingEncoder, TargetEncoder
from sklearn.model_selection import KFold
import numpy as np

In [ ]:
df = pd.read_csv('../data/ml_ozon_сounterfeit_train.csv')
df.describe()

,id,resolution,rating_1_count,rating_2_count,rating_3_count,rating_4_count,rating_5_count,comments_published_count,photos_published_count,videos_published_count,...,ExemplarReturnedCountTotal30,ExemplarReturnedCountTotal90,ExemplarReturnedValueTotal7,ExemplarReturnedValueTotal30,ExemplarReturnedValueTotal90,ItemVarietyCount,ItemAvailableCount,seller_time_alive,ItemID,SellerID
count,1000.000000,1000.000000,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,...,994.000000,995.000000,986.000000,994.000000,995.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000
mean,212433.058000,0.018000,2.497297,0.827027,2.024324,2.416216,23.056757,16.800000,6.329730,0.610811,...,32.357143,91.104523,672.076090,900.190121,1024.941139,4.066000,4.066000,682.18000,104261.469000,2542.602000
std,129061.988652,0.133018,7.477971,3.074914,10.375182,10.958644,79.451879,85.072824,19.599221,3.659622,...,68.946103,161.648111,381.029088,302.609676,265.625694,6.571212,6.571212,399.08901,62995.706182,2977.425994
min,370.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,10.00000,191.000000,5.000000
25%,101925.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,3.000000,10.000000,621.433319,876.185969,967.886157,1.000000,1.000000,351.75000,50263.000000,315.000000
50%,204755.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,1.000000,0.000000,0.000000,...,17.000000,46.000000,834.808188,982.464632,1078.191995,2.000000,2.000000,649.00000,100237.500000,1045.000000
75%,322147.000000,0.000000,2.000000,1.000000,1.000000,1.000000,10.000000,5.000000,4.000000,0.000000,...,41.000000,120.500000,920.311331,1056.261435,1163.536598,4.000000,4.000000,988.00000,158566.750000,4029.000000
max,454708.000000,1.000000,90.000000,46.000000,180.000000,189.000000,1067.000000,1303.000000,204.000000,65.000000,...,1058.000000,2138.000000,1303.531356,1424.062031,1496.049407,31.000000,31.000000,1923.00000,220617.000000,12386.000000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 45 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            10000 non-null  int64  
 1   resolution                    10000 non-null  int64  
 2   brand_name                    5762 non-null   object 
 3   description                   8894 non-null   object 
 4   name_rus                      10000 non-null  object 
 5   CommercialTypeName4           10000 non-null  object 
 6   rating_1_count                3226 non-null   float64
 7   rating_2_count                3226 non-null   float64
 8   rating_3_count                3226 non-null   float64
 9   rating_4_count                3226 non-null   float64
 10  rating_5_count                3226 non-null   float64
 11  comments_published_count      3226 non-null   float64
 12  photos_published_count        3226 non-null   float64
 13  vi

In [4]:
df.columns

Index(['id', 'resolution', 'brand_name', 'description', 'name_rus',
       'CommercialTypeName4', 'rating_1_count', 'rating_2_count',
       'rating_3_count', 'rating_4_count', 'rating_5_count',
       'comments_published_count', 'photos_published_count',
       'videos_published_count', 'PriceDiscounted', 'item_time_alive',
       'item_count_fake_returns7', 'item_count_fake_returns30',
       'item_count_fake_returns90', 'item_count_sales7', 'item_count_sales30',
       'item_count_sales90', 'item_count_returns7', 'item_count_returns30',
       'item_count_returns90', 'GmvTotal7', 'GmvTotal30', 'GmvTotal90',
       'ExemplarAcceptedCountTotal7', 'ExemplarAcceptedCountTotal30',
       'ExemplarAcceptedCountTotal90', 'OrderAcceptedCountTotal7',
       'OrderAcceptedCountTotal30', 'OrderAcceptedCountTotal90',
       'ExemplarReturnedCountTotal7', 'ExemplarReturnedCountTotal30',
       'ExemplarReturnedCountTotal90', 'ExemplarReturnedValueTotal7',
       'ExemplarReturnedValueTotal30', '

In [6]:
from pymorphy3 import MorphAnalyzer
import re
from concurrent.futures import ThreadPoolExecutor

morph = MorphAnalyzer(lang='ru')

def lemmatize_batch(texts):
    results = []
    for text in texts:
        if not text:
            results.append('')
            continue
        
        tokens = re.findall(r'\b[а-яё]+\b', text.lower())
        lemmas = []
        for token in tokens:
            if len(token) > 1:
                parsed = morph.parse(token)[0]
                lemmas.append(parsed.normal_form)
        results.append(' '.join(lemmas))
    
    return results

def clean_column_optimized(series, batch_size=1000):
    series = series.fillna('')
    
    cleaned = series.apply(lambda x: BeautifulSoup(str(x), 'html.parser').get_text())\
                   .apply(lambda x: unicodedata.normalize('NFKC', x))\
                   .str.lower()\
                   .str.replace(r'[^\w\s]', ' ', regex=True)\
                   .str.replace(r'\s+', ' ', regex=True)\
                   .str.strip()
    
    texts = cleaned.tolist()
    lemmatized_texts = []
    
    with ThreadPoolExecutor() as executor:
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i + batch_size]
            lemmatized_batch = lemmatize_batch(batch)
            lemmatized_texts.extend(lemmatized_batch)
    
    return pd.Series(lemmatized_texts, index=series.index)

df['description'] = clean_column_optimized(df['description'])

/tmp/ipykernel_25562/2618008016.py:27: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  cleaned = series.apply(lambda x: BeautifulSoup(str(x), 'html.parser').get_text())\


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import nltk
from nltk.corpus import stopwords

try:
    russian_stopwords = stopwords.words('russian')
except:
    nltk.download('stopwords')
    russian_stopwords = stopwords.words('russian')

def simple_tfidf_processing(df, text_column='description'):
    
    texts = df[text_column].fillna('').tolist()
    
    char_tfidf = TfidfVectorizer(
        analyzer='char',
        ngram_range=(3, 5),
        max_features=10000,
        min_df=2
    )
    
    X_char = char_tfidf.fit_transform(texts)
    char_features = [f'char_{f}' for f in char_tfidf.get_feature_names_out()]
    
    word_tfidf = TfidfVectorizer(
        analyzer='word', 
        ngram_range=(1, 3),
        max_features=10000,
        min_df=2,
        stop_words=russian_stopwords,
        lowercase=True,
        token_pattern=r'\b[а-яёa-z]+\b'
    )
    
    X_word = word_tfidf.fit_transform(texts)
    word_features = [f'word_{f}' for f in word_tfidf.get_feature_names_out()]
    
    char_df = pd.DataFrame(X_char.toarray(), columns=char_features, index=df.index)
    word_df = pd.DataFrame(X_word.toarray(), columns=word_features, index=df.index)
    
    result_df = pd.concat([df, char_df, word_df], axis=1)
    
    print(f"{len(char_features)} символьных фич")
    print(f"{len(word_features)} словных фич")
    
    return result_df, char_features + word_features

df, feature_names = simple_tfidf_processing(df)
print('Description is ready')
df, _ = simple_tfidf_processing(df, text_column='name_rus')

10000 символьных фич
8097 словных фич
Description is ready
10000 символьных фич
1831 словных фич


In [9]:
list(df_processed.columns)

['id',
 'resolution',
 'brand_name',
 'description',
 'name_rus',
 'CommercialTypeName4',
 'rating_1_count',
 'rating_2_count',
 'rating_3_count',
 'rating_4_count',
 'rating_5_count',
 'comments_published_count',
 'photos_published_count',
 'videos_published_count',
 'PriceDiscounted',
 'item_time_alive',
 'item_count_fake_returns7',
 'item_count_fake_returns30',
 'item_count_fake_returns90',
 'item_count_sales7',
 'item_count_sales30',
 'item_count_sales90',
 'item_count_returns7',
 'item_count_returns30',
 'item_count_returns90',
 'GmvTotal7',
 'GmvTotal30',
 'GmvTotal90',
 'ExemplarAcceptedCountTotal7',
 'ExemplarAcceptedCountTotal30',
 'ExemplarAcceptedCountTotal90',
 'OrderAcceptedCountTotal7',
 'OrderAcceptedCountTotal30',
 'OrderAcceptedCountTotal90',
 'ExemplarReturnedCountTotal7',
 'ExemplarReturnedCountTotal30',
 'ExemplarReturnedCountTotal90',
 'ExemplarReturnedValueTotal7',
 'ExemplarReturnedValueTotal30',
 'ExemplarReturnedValueTotal90',
 'ItemVarietyCount',
 'ItemAvailab

In [14]:
# Target Encoding
kf = KFold(n_splits=5, shuffle=True, random_state=42)
df['CommercialTypeName4_target'] = np.nan

for train_idx, val_idx in kf.split(df):
    means = df.iloc[train_idx].groupby('CommercialTypeName4')['resolution'].mean()
    df.iloc[val_idx, df.columns.get_loc('CommercialTypeName4_target')] = \
        df.iloc[val_idx]['CommercialTypeName4'].map(means).fillna(df['resolution'].mean())

In [15]:
# Hashing Encoding
df['brand_name'] = df['brand_name'].fillna('MISSING')

hasher = HashingEncoder(cols=['brand_name'], n_components=12)
df_hashed = hasher.fit_transform(df[['brand_name']])

df = pd.concat([df, df_hashed], axis=1)

In [16]:
df = df.drop(columns=['CommercialTypeName4', 'brand_name', 'description', 'name_rus'])

In [17]:
df

,id,resolution,rating_1_count,rating_2_count,rating_3_count,rating_4_count,rating_5_count,comments_published_count,photos_published_count,videos_published_count,...,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11
0,159385,0,6.0,4.0,4.0,3.0,32.0,3.0,6.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,288616,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
2,108090,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,415607,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
4,332391,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,329294,0,10.0,8.0,9.0,2.0,6.0,13.0,4.0,0.0,...,0,0,0,0,0,0,0,0,1,0
996,441110,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
997,382241,0,1.0,1.0,0.0,3.0,41.0,47.0,11.0,0.0,...,0,0,0,0,0,0,0,1,0,0
998,155636,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
